In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from IPython.display import display, clear_output

# Constants

In [2]:
initial_board = np.zeros((6, 7), int)

# Game

In [3]:
class State():
    def __init__(self, board: np.array, player: int = np.random.randint(1, 3), game_over: bool = False):
        self.board = board
        self.player = player
        # self.actions = self.get_actions()
        self.game_over = game_over
        
        
    def get_actions(self) -> list:
        '''
        creates a list of column indices that are available for turns
        
        return:
            list of column indices
        '''
        
        # get all columns where the top row is 0 / empty
        actions = np.where(self.board[0,:] == 0)
        
        return list(actions[0])
    
    
    def perform_action(self, column: int):
        '''
        adds a the player's coin value to the given columns on the board
        
        Args:
            column: Index of the column that is used for this turn
        '''
        
        # set height 
        height = self.board.shape[0]
        
        # loop backwards from height to 0
        for row in range(height -1, -1, -1):
            
            # add player number to position if it is empty (0) and break loop
            if self.board[row, column] == 0:
                
                new_board = self.board.copy()
                new_board[row, column] = self.player
                
                game_over = self.check_win(new_board, row, column)
                
                player = self.change_player()
                    
                return State(new_board, player, game_over) 
        
        print("Error while perform_action")
        return None
    
    
    def check_win(self, board: np.array, row: int, column: int) -> bool:
        '''
        checks win conditions for current player by cheking straight and diagonal lines through the last used position
        
        Args:
            row: row index of the position of the last action
            column: column index of the position of the last action
        
        return:
            True if game is won, else False
        '''
        
        # set possible lines through given position on board defined by row and column
        lines = {
            "horizontal": board[row, :],
            "vertical": board[:, column],
            "diagonal_1": np.diagonal(board, offset= column - row),
            "diagonal_2": np.diagonal(np.flipud(board), offset = column - (board.shape[0] - row - 1)),
            }
        
        # loop through possible lines and check for positions marked with current player's number
        for key in lines.keys():
            # set counter and loop through line
            counter = 0
            for position in lines[key]:
                
                # increase counter if player's number is found
                if position == self.player:
                    counter += 1
                    
                    # return true and end loop if counter reaches 4
                    if counter >= 4:
                        return True
                    
                # reset counter if another value than the player's number is found 
                else:
                    counter = 0
                    
        return False    
    
    
    def change_player(self) -> int:
        '''
        Sets the value of the player to 2 if 1 is given and vice versa
        
        return:
            number of new player
        '''
        
        if self.player == 1:
            player = 2
        else:
            player = 1
        
        return player

# Tree

In [4]:
class Node():
    def __init__(self, parent, state: State, former_action: int):
        self.state = state
        self.parent = parent
        self.edges = self.state.get_actions()
        self.children  = []
        self.times_tested = 0
        self.times_won = 0
        self.former_action = former_action
        self.explored = False
        
    def get_children(self):
        '''
        creates a list of Nodes that are descendants of this Node
        '''
        # get all possible actions
        actions = self.state.get_actions()
        
        # create a Node object for each action and add it to the list of children
        for action in actions:  
            self.children.append(Node(self, self.state.perform_action(action), action))
        

# Utils

In [5]:
def show_board(node: Node, delay: float):
    
    plt.style.use('dark_background')
    fig, ax = plt.subplots()
    ax.imshow(node.state.board)
    if node.state.game_over:
        ax.set_title(f"GAME OVER\nplayer {node.state.player} lost")
    display(fig)
    clear_output(wait=True)
    plt.pause(delay)

# Selection

In [6]:
def normalize_vector(vector:np.array) -> np.array:
    '''
    normalizes a vector based von min max scaling
    
    Args:
        vector: Input Vector as numpy.array
        
    return:
        normalized version of input vector
    '''
    
    # get min and max values of vector
    min = np.amin(vector)
    max = np.amax(vector)
    
    # calculate normalized vector
    norm_vector = (vector - min)/(max - min)
    
    return norm_vector
    

def upper_confidence_boundary(values: np.array, n_tests: np.array, step: int, c: float = 0.5) -> int:
    '''
    chooses an index for a list of nodes based on upper confidence boundary calculation
    
    Args:
        values: normalized numpy.array of node values
        n_tests: numpy.array keeping track of how often a node was previously tested
        step: time step for this calculation
        c: constant to adjust the amount of exploration/ exploitation for this calculatoin
        
    return:
        index of chosen Node 
    '''
    
    # calculate ucb value
    ucb_values = values + c * np.sqrt(np.log(step)/n_tests)
    
    # get index of highest value
    node_index = np.argmax(ucb_values)

    ################
    # UCB mit UCT ersetzen
    ################
    return node_index



def select_node(node: Node) -> Node:
    '''
    selects Node to test/ simulate based on their previous outcomes and amount of exploration.
    
    Args:
        root: Starting Node for selection
        
    return:
        Node to elaborate
    '''
    
    # get list of children that are not completely explored
    children = [child for child in node.children if not child.explored]
    
    # if there are no unexplored children left repeat selection one level above
    if not children:
        
        # mark current node node as explored
        node.explored = True
        
        # go to selection at level above or stop if root is reached
        if node.parent:
            select_node(node.parent)
        else:
            print("tree fully explored")
            return None
     ###########
     # Maybe mark explored nodes in Backpropagation phase
     ###########
            
    else:
    
        # get the exploration values of root's children
        n_tests = np.array([node.times_tested for node in children])
        
        # consider untested nodes first
        if 0 in n_tests:
            
            # pick random if multiple unvisited exist
            random_index = random.choice(np.where(n_tests == 0)[0])
            new_node = children[random_index]
            
            return new_node
        
        # choose between tested nodes using upper confidence boundary
        else:
            # get normalized values of children nodes
            values = [node.times_won for node in children]
            norm_values = normalize_vector(np.array(values))
            
            # choose node with upper confidence boundary
            
            ###############
            # Include choices of opponent /minimize uct if needed
            ################
            
            node_index = upper_confidence_boundary(norm_values, np.array(n_tests), 1)
            new_node = children[node_index]
            
            # repeat selection if node has children
            if new_node.children:
                select_node(new_node)
            
            # otherwise return node
            else:
                return new_node


# Expansion

In [7]:
def expand_tree(node: Node) -> Node:
    
    if node.times_tested > 0:
        node.get_children()
        
        if node.children:
            new_node = random.choice(node.children)
            
            return new_node
        
        else:
            node.explored = True
            ########
            # check/ handover if simulation is necessary
            #######
            # explored maybe added in simulation phase
            #######
            return node
        
    else:
        return node

# Simulation

In [8]:
def simulate(node: Node) -> bool:
    '''
    Uses random choices to simulate the outcome of a game
    
    Args:
        Node: Defines the starting situation of the simulation
        
    Return:
        True if active player in given node wins, False if not or draw
    '''
    
    # define relevant Player for rating
    active_player = node.state.player

    #######
    # work with state instead of node
    #######
    
    
    # add children if necessary
    if not node.children:
        node.get_children()
    
    # loop while children exist and game is not over
    while node.children != [] and node.state.game_over is False:
        
        # pick random child node
        node = random.choice(node.children)
        
        # add further child nodes
        node.get_children()
        
        show_board(node, 0.01)
        
    # return losing player
    return node.state.player
        

# Backpropagation

In [9]:
# def backpropagate(node: Node, loser: int) -> None:

#     node.times_tested += 1
#     if node.state.player != loser:
#         node.times_won += 1
    
#     if node.parent:
#         backpropagate(node.parent, loser)
#     else:
#         return None

In [10]:
def backpropagate(node: Node, loser: int) -> None:
    
    while node.parent:
        
        node.times_tested += 1
        if node.state.player != loser:
            node.times_won += 1
            
        node = node.parent
        
    return None

# Monte Carlo Tree Search

In [11]:
def mcts(root: Node) -> int:
    
    t = 0
    tree_search = True
    
    while t < 100 and tree_search:
        if select_node(root):
            node = select_node(root)
            node = expand_tree(node)
            loser = simulate(node)
            backpropagate(node, loser)
            
            t += 1
        
        else:
            tree_search = False
    
    action = choose_action(root)

    return action
    
def choose_action(root: Node) -> int:
    
    indices = np.array([]) 
    values = np.array([])
    n_tests = np.array([])
    
    for i, child in enumerate(root.children):
        if child.times_tested > 0:
            indices = np.append(indices, i)
            values = np.append(values, child.times_won)
            n_tests = np.append(n_tests, child.times_tested)

    
    mean_values = values/ n_tests
    max_value = np.argmax(mean_values)
    
    index = int(indices[max_value])
    best_node = root.children[index]
    
    chosen_action = best_node.former_action

    return chosen_action


# Test

In [14]:
initial_board = np.zeros((6, 7), int)
state = State(board = initial_board)

state.get_actions()
state.board


array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]])

In [13]:
# print(f"current player: {node.state.player}")
# print(f"times tested: {node.times_tested}")
# print(f"times won: {node.times_won}")


a = [1, 0, 7, 5, 0, 56, 3]

[(i, i+2) for i, num in enumerate(a) if num > 0]

[(0, 2), (2, 4), (3, 5), (5, 7), (6, 8)]